In [1]:
%load_ext lab_black

import errno
import json
import os
import shutil
import stat
from copy import deepcopy
from datetime import datetime
from enum import Enum
from os import listdir
from os.path import isfile, join
from pathlib import Path
from random import random

import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import tensorflow as tf
from keras import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.layers import *
from PIL import Image, ImageDraw
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tqdm.notebook import tqdm

# tf.debugging.set_log_device_placement(True)
plt.rcParams["figure.figsize"] = [40, 20]
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

Num GPUs Available:  1


In [2]:
from ArrayLoader import *
from augmentation import *
from CWDataset import *
from datautils import *

In [3]:
original_path = Path("data/CW/original")
generated_path = Path("data/CW/generatedBig")

In [4]:
convert_wepb(original_path)

Found 0 webp files. Converting


In [5]:
real_shape = (544, 1280, 3)
save_shape = (544 // 4, 1280 // 4, 3)
augmented_samples = 20

In [6]:
def augment():
    labels_json = dict()
    original_images = list(original_path.glob("*.jpeg"))
    for image_path in tqdm(original_images):
        #   print(image_path)
        img, labels = load_image(image_path)
        res_im, res_la = generate_augmentations_for_image(
            img, labels, augmented_samples
        )

        for i in range(len(res_im)):
            new_path = generated_path.joinpath(
                image_path.name.split(".")[0] + "_generated_" + str(i) + ".jpeg"
            )
            # print(new_path)
            cv2.imwrite(str(new_path), res_im[i])  # res_la[i])
            labels_json[new_path.name] = {"labels": np.asarray(res_la[i]).tolist()}

        del res_im
        del res_la

    with open(str(generated_path.joinpath("labels.json")), "w") as f:
        json.dump(labels_json, f)

In [7]:
augment()

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 143/143 [00:43<00:00,  3.29it/s]


In [8]:
dataset = CWDataset(generated_path)

Train 2249
Test  296
Vali  295
